In [53]:
"""
The script below extract the stock prices for the past 6 months using the Quandl API.
The data retrieved and the computed fluctuations are stored automatically into separte text file"""

import pandas as pd
import quandl
import numpy as np
import pickle
import os
from datetime import date
from datetime import datetime
import sqlite3
quandl.ApiConfig.api_key = '4eneTCPvj2z6youDnQcb' 

def getStockDir():
    cwd = os.getcwd()
    parentDir = os.path.dirname(cwd)
    outputPath = makeDirectory(parentDir , "StockData") 
    return outputPath

def makeDirectory(folder_name1,sub_folder):
    directory = os.path.join(folder_name1 , sub_folder)
    if not os.path.exists(directory):
            os.makedirs(directory)
    return directory

def getOutputDir(folderName,fileName):
    d = outputPath + "/%s"%(fileName)
    return d

def convertDate(s):
    return datetime.strptime(s,'%Y-%m-%d') 

def getWeek(d):
    StartDate = date(2013,1,7)
    endDate = d
    ans = int((end - StartDate).days/7)
    return ans

def getDay(s): 
    weekNo = date.weekday(s)
    return weekNo

def computeDailyChange(data1):
    i = 0   
    dataSet = [] 
    fluctuations = [] 
   
    for stock in data1:
        dataSet.append(stock)

 # print(dataSet)

    while i < (len(dataSet)-1):
        val = ((dataSet[i+1] - dataSet[i])/dataSet[i])*100
        fluctuations.append("%0.2f" % val)
    #	print(fluctuations)
    #	print("\n")
        i = i + 1 

    return fluctuations 	

def computeDayNumber(data):
    dayZero = date(2016,1,4)
    day = []
    for d in data:
        tmp = d.date() - dayZero
        day.append("Day " + str(tmp.days))
    day.pop(0)
    return day

def savePickleFile(d,varList):
    with open(d, 'wb') as f: 
        pickle.dump(varList, f)
    f.close()
def loadPickleFile(d):
    with open(d,'rb') as f:  # Python 3: open(..., 'rb')
        varList = pickle.load(f)
    f.close()
    return varList 

def getFluc(data):
    a= np.array(data['close'][1:])
    b = np.array(data['close'][:-1])
    c=(a-b)/b*100
    d=data["date"][1:].astype(str)
    
    tmp = np.column_stack((d,c))
    return (np.column_stack((d,c)))
   
def connectDB():
    cwd = os.getcwd()
    parentDir = os.path.dirname(cwd)
    sqlite_file = parentDir+"/mydatabase.db"
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    command="CREATE TABLE IF NOT EXISTS Stock(date TEXT, stockName TEXT, open INTEGER, close INTEGER, volume INTEGER, change INTEGER, UNIQUE(date, stockName) ON CONFLICT REPLACE);"
    c.execute(command)
    return conn

def insertToDB(data,predicted,c,company):
    #c is connector SQL
    table_name = "Stock"
    for i in range(len(data)):
        values = (predicted[i][0],company,data['open'].values[i],
                  data['close'].values[i],data['volume'].values[i],predicted[i][1])
        c.execute("INSERT INTO {tn} VALUES(?,?,?,?,?,?)".format(tn=table_name),values)

def disconnectDB(conn):
    conn.commit()
    conn.close()
           
#pre-defined company list - 201 stocks
companies=["AMZN","AAPL","FB","GOOGL","TWTR","MU","PGR","NFLX","GS","JNJ","DAL","BAC","CSCO","INTC","F","WFC","KMI","CVS","CLX","ALB","MCHP","EQR","SPG","M","CF","KHC","MDLZ","PG","MSFT","TSLA","CHKP","MAC","ORCL","AAON","FOX","MAA","DIS","C","CRI","SAFT","GES","SPLK","CRI","SN","SXT","ITC","CROX","FAST","LOGM","CAH","V","UNH","KO","GS","WMT","BP","MRK","VZ","UTX","TRV","DIS","BA","NKE","MCD","JPM","GE","SEM","CVX","CAT","AXP","IBM","ALKS","BK","CELG","CHTR","CTXS","COST","DLTR","DISH","EBAY","ESRX","HAS","MAR","MAT","PYPL","SIRI","SBUX","SYMC","TSCO","ULTA","VIAB","VOD","VRTX","WDC","WBA","XRAY","ORLY","STX","WYNN","XLNX","PCAR","IDXX","CBT","COG","CAMP","CWT","CALX","CPT","CCBG","COF","CAH","DDR","FFG","AGM","FDX","GPS","GLOG","IT","GD","GIS","GM","GGP","HALL","ISSC","IBP","ICE","ICPT","IGT","XON","JCP","JBL","JACK","KTWO","K","KEG","KNX","LADR","LVS","LTM","TREE","LSI","LECO","LNC","M","MRO","MPX","VAC","MA","MAT","MXWL","MDR","MED","MCC","MGM","KORS","MSTR","MSEX","MPO","MTX","MS","MORN","MUR","NANO","MC","P","PEI","PE","PWR","RRD","RDN","RL","RNR","SPGI","WM","FRO","FSLR","EZPW","KSS","GES","ADI","ADBE","ADSK","ADP","AKAM","ALGN","ATVI","CHTR","DISCA","EXPE","GILD","MAT","TMUS","AAL","AXP","UTX","A","T","AVGO","CBS","CREE","DOV","GRMN","BDX","ARNA","CBS","NBIX","SBGI","DO","AMBA","NVDA","AMD"]
# companies=["AAPL","AMZN"]
allFluctuations= []
approved_companies=[]

count = 0
companiesFluc = []


if (os.path.isfile(getStockDir()+"/stockConfig.pkl")):
    oldest,earliest = loadPickleFile(getStockDir()+"/stockConfig.pkl")
    oldest=earliest
    earliest = datetime.now().strftime('%Y-%m-%d')
    savePickleFile(getStockDir()+"/stockConfig.pkl",[oldest,earliest])
else:
    oldest = '2012-01-01'
    t = datetime.now()
    earliest=t.strftime('%Y-%m-%d')
    savePickleFile(getStockDir()+"/stockConfig.pkl",[oldest,earliest])
            
conn = connectDB()
for company in companies:
    # fetch the company data
    data = quandl.get_table('WIKI/PRICES', ticker = company, 
                        qopts = { 'columns': ['date', 'open', 'close', 'volume'] }, 
                       date = { 'gte': oldest, 'lte': earliest })

    if len(data) == 0:
        print ('Data for %s is not available'%company)
    else:
        print ('Data for %s fetched'%company)
        # computation of the daily fluctuations
        sendToDB = data[1:]
        predicted = getFluc(data)
        insertToDB(sendToDB,predicted,conn.cursor(),company)
disconnectDB(conn)
        #print(dailyFluctuations)
        # add to the list of fluctuation of all companies
#         allFluctuations.append(dailyFluctuations)
#                 #print(allFluctuations)
#         # write the data to a text file
#         companiesData_filename = os.path.join(companies_directory, company + ".txt")
#         companiesData_output_file=open(companiesData_filename,'w')
#         data.to_string(companiesData_output_file,index = False)

        

#print(approved_companies)
# aggregateDailyFluctuations(allFluctuations,approved_companies)


Data for AMZN fetched
Data for AAPL fetched
Data for FB fetched
Data for GOOGL fetched
Data for TWTR fetched
Data for MU fetched
Data for PGR fetched
Data for NFLX fetched
Data for GS fetched
Data for JNJ fetched
Data for DAL fetched
Data for BAC fetched
Data for CSCO fetched
Data for INTC fetched
Data for F fetched
Data for WFC fetched
Data for KMI fetched
Data for CVS fetched
Data for CLX fetched
Data for ALB fetched
Data for MCHP fetched
Data for EQR fetched
Data for SPG fetched
Data for M fetched
Data for CF fetched
Data for KHC fetched
Data for MDLZ fetched
Data for PG fetched
Data for MSFT fetched
Data for TSLA fetched
Data for CHKP fetched
Data for MAC fetched
Data for ORCL fetched
Data for AAON fetched
Data for FOX fetched
Data for MAA fetched
Data for DIS fetched
Data for C fetched
Data for CRI fetched
Data for SAFT fetched
Data for GES fetched
Data for SPLK fetched
Data for CRI fetched
Data for SN fetched
Data for SXT fetched
Data for ITC fetched
Data for CROX fetched
Data fo

-0.8490197173658103

-0.8490197173658103